# Question 1 : Explication de type (1-1) pour la Comparaison x ≻ y

## Objectif
Formuler un programme d'optimisation linéaire qui calcule une explication de type (1-1) de la comparaison x ≻ y si elle existe, et retourne un certificat de non-existence sinon.

## Approche et Formulation Mathématique

### 1. Données et Paramètres
- **x, y** : vecteurs de notes des deux candidats pour chaque cours i
- **w** : vecteur des poids de chaque cours
- **contributions(i)** = w[i] × (x[i] - y[i]) : contribution du cours i à la somme pondérée

### 2. Ensembles
- **pros(x,y)** : ensemble des cours i où x[i] > y[i] (contributions > 0)
- **cons(x,y)** : ensemble des cours i où x[i] < y[i] (contributions < 0)
- **neutral(x,y)** : ensemble des cours i où x[i] = y[i] (contributions = 0)

### 3. Formulation du PL

**Variables de décision:**
- Pour chaque paire (P, C) où P ∈ pros(x,y) et C ∈ cons(x,y) :
  - x_{P,C} ∈ {0, 1} : égal à 1 si la paire (P, C) est sélectionnée comme trade-off

**Contraintes:**
1. Chaque cours C ∈ cons(x,y) doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Chaque cours P ∈ pros(x,y) peut être appairé au plus une fois :
   $$\sum_{C \in \text{cons}(x,y)} x_{P,C} \leq 1 \quad \forall P \in \text{pros}(x,y)$$

3. Chaque trade-off sélectionné doit être valide (contribution(P) + contribution(C) > 0) :
   $$(\text{contribution}(P) + \text{contribution}(C)) \cdot x_{P,C} > 0$$
   
   Ou de façon linéaire (si toutes les paires positives) :
   $$\text{contribution}(P) + \text{contribution}(C) > 0 \quad \text{pour chaque paire sélectionnée}$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P,C} x_{P,C}$
- Ou maximiser pour chercher une explication quelconque : $\max \sum_{P,C} x_{P,C}$

**Certificat de non-existence:**
Si le PL est infaisable (aucune solution), il n'existe pas d'explication (1-1).

In [9]:
import numpy as np
from gurobipy import *

# Données du problème (exemple du prologue)
courses = ['Anatomie', 'Biologie', 'Chirurgie', 'Diagnostic', 'Epidemiologie', 'Forensic', 'Genetique']
x_notes = np.array([85, 81, 71, 69, 75, 81, 88])
y_notes = np.array([81, 81, 75, 63, 67, 88, 95])
weights = np.array([8, 7, 7, 6, 6, 5, 6])

# Calcul des contributions
contributions = weights * (x_notes - y_notes)

# Affichage des données
print("=" * 70)
print("DONNÉES DU PROBLÈME")
print("=" * 70)
print("\nCours\t\t  x_i  y_i  w_i  (x_i - y_i)  contribution")
print("-" * 70)
for i, course in enumerate(courses):
    diff = x_notes[i] - y_notes[i]
    contrib = contributions[i]
    print(f"{course:20} {x_notes[i]:3d}  {y_notes[i]:3d}  {weights[i]:2d}    {diff:3d}         {contrib:4d}")

# Calcul des moyennes pondérées
x_score = np.sum(weights * x_notes) / np.sum(weights)
y_score = np.sum(weights * y_notes) / np.sum(weights)
total_contribution = np.sum(contributions)

print("\n" + "=" * 70)
print(f"Moyenne pondérée de x : {x_score:.2f}")
print(f"Moyenne pondérée de y : {y_score:.2f}")
print(f"Somme totale des contributions : {total_contribution}")
print("=" * 70)

# Identification des ensembles pros, cons et neutral
pros = [i for i in range(len(courses)) if contributions[i] > 0]
cons = [i for i in range(len(courses)) if contributions[i] < 0]
neutral = [i for i in range(len(courses)) if contributions[i] == 0]

print("\nEnsembles:")
print(f"pros(x,y) = {{{', '.join([courses[i] for i in pros])}}}")
print(f"cons(x,y) = {{{', '.join([courses[i] for i in cons])}}}")
print(f"neutral(x,y) = {{{', '.join([courses[i] for i in neutral])}}}")

print("\nContributions détaillées:")
for i in pros:
    print(f"  {courses[i]:20} (PRO)  : +{contributions[i]}")
for i in cons:
    print(f"  {courses[i]:20} (CONS) : {contributions[i]}")
for i in neutral:
    print(f"  {courses[i]:20} (NEUTRE): {contributions[i]}")

DONNÉES DU PROBLÈME

Cours		  x_i  y_i  w_i  (x_i - y_i)  contribution
----------------------------------------------------------------------
Anatomie              85   81   8      4           32
Biologie              81   81   7      0            0
Chirurgie             71   75   7     -4          -28
Diagnostic            69   63   6      6           36
Epidemiologie         75   67   6      8           48
Forensic              81   88   5     -7          -35
Genetique             88   95   6     -7          -42

Moyenne pondérée de x : 78.69
Moyenne pondérée de y : 78.44
Somme totale des contributions : 11

Ensembles:
pros(x,y) = {Anatomie, Diagnostic, Epidemiologie}
cons(x,y) = {Chirurgie, Forensic, Genetique}
neutral(x,y) = {Biologie}

Contributions détaillées:
  Anatomie             (PRO)  : +32
  Diagnostic           (PRO)  : +36
  Epidemiologie        (PRO)  : +48
  Chirurgie            (CONS) : -28
  Forensic             (CONS) : -35
  Genetique            (CONS) : -42
  Biolo

In [8]:
# Énumération de tous les trade-offs (1-1) valides
print("\n" + "=" * 70)
print("ÉNUMÉRATION DE TOUS LES TRADE-OFFS (1-1) VALIDES")
print("=" * 70)

valid_tradeoffs = []
for p_idx in pros:
    for c_idx in cons:
        sum_contrib = contributions[p_idx] + contributions[c_idx]
        if sum_contrib > 0:
            valid_tradeoffs.append((p_idx, c_idx, sum_contrib))
            print(f"({courses[p_idx]}, {courses[c_idx]}): {contributions[p_idx]:4d} + ({contributions[c_idx]:4d}) = {sum_contrib:4d} ✓")

print(f"\nNombre total de trade-offs (1-1) valides : {len(valid_tradeoffs)}")



ÉNUMÉRATION DE TOUS LES TRADE-OFFS (1-1) VALIDES
(Anatomie, Chirurgie):   32 + ( -28) =    4 ✓
(Diagnostic, Chirurgie):   36 + ( -28) =    8 ✓
(Diagnostic, Forensic):   36 + ( -35) =    1 ✓
(Epidemiologie, Chirurgie):   48 + ( -28) =   20 ✓
(Epidemiologie, Forensic):   48 + ( -35) =   13 ✓
(Epidemiologie, Genetique):   48 + ( -42) =    6 ✓

Nombre total de trade-offs (1-1) valides : 6


## Formulation du Problème d'Optimisation Linéaire en Détail

### Variables de décision
Pour chaque trade-off (1-1) valide (P, C) :
- **x[P,C]** ∈ {0, 1} : égal à 1 si le trade-off est sélectionné, 0 sinon

### Contraintes
1. **Couverture complète des cons** : Chaque cours C ∈ cons(x,y) doit être appairé exactement une fois
   - Cela garantit que chaque reproche sera contré par un avantage

2. **Limitation des pros** : Chaque cours P ∈ pros(x,y) peut être utilisé au maximum une fois
   - Cela garantit que chaque argument ne peut servir qu'une fois

### Objectif
**Minimiser** la longueur de l'explication : minimiser le nombre de paires (P, C) sélectionnées

### Cas d'infaisabilité
Si le problème n'admet pas de solution, cela signifie qu'il n'existe pas d'explication (1-1), ce qui serait un certificat que x ≯ y ou que le comparaison est invalide.

In [6]:
# Implémentation du PL avec Gurobi

print("\n" + "=" * 70)
print("RÉSOLUTION DU PROGRAMME D'OPTIMISATION LINÉAIRE")
print("=" * 70)

# Création du modèle
m = Model("Explication_1-1")

# Variables de décision : x[p,c] pour chaque trade-off valide
x_vars = {}
for p_idx, c_idx, _ in valid_tradeoffs:
    x_vars[(p_idx, c_idx)] = m.addVar(vtype=GRB.BINARY, 
                                       name=f"x_{courses[p_idx]}_{courses[c_idx]}")

m.update()

# Contrainte 1 : Chaque cours C ∈ cons doit être appairé exactement une fois
for c_idx in cons:
    relevant_pairs = [(p, c) for p, c, _ in valid_tradeoffs if c == c_idx]
    if relevant_pairs:
        m.addConstr(
            quicksum(x_vars[(p, c)] for p, c in relevant_pairs) == 1,
            name=f"cover_{courses[c_idx]}"
        )

# Contrainte 2 : Chaque cours P ∈ pros peut être utilisé au maximum une fois
for p_idx in pros:
    relevant_pairs = [(p, c) for p, c, _ in valid_tradeoffs if p == p_idx]
    if relevant_pairs:
        m.addConstr(
            quicksum(x_vars[(p, c)] for p, c in relevant_pairs) <= 1,
            name=f"limit_{courses[p_idx]}"
        )

# Objectif : Minimiser le nombre de trade-offs
m.setObjective(quicksum(x_vars.values()), GRB.MINIMIZE)

# Configuration et résolution
m.params.outputflag = 0
m.optimize()

print(f"\nStatut de résolution : ", end="")
if m.status == GRB.OPTIMAL:
    print("OPTIMAL ✓")
elif m.status == GRB.INFEASIBLE:
    print("INFAISABLE ✗")
    print("\n⚠️ CERTIFICAT DE NON-EXISTENCE")
    print("Aucune explication de type (1-1) n'existe pour cette comparaison.")
    print("Cela signifie qu'il est impossible de justifier x ≻ y avec des trade-offs (1-1).")
elif m.status == GRB.UNBOUNDED:
    print("NON BORNÉ")
else:
    print(f"AUTRE ({m.status})")

# Affichage des résultats si optimal
if m.status == GRB.OPTIMAL:
    print(f"\n✓ UNE EXPLICATION DE TYPE (1-1) EXISTE")
    print(f"Longueur minimale de l'explication : {int(m.objVal)} trade-offs")
    print("\nExplication détaillée:")
    print("-" * 70)
    
    explanation_pairs = []
    for (p_idx, c_idx), var in x_vars.items():
        if var.X > 0.5:  # Si la variable est sélectionnée (valeur 1)
            p_contrib = contributions[p_idx]
            c_contrib = contributions[c_idx]
            total = p_contrib + c_contrib
            explanation_pairs.append((p_idx, c_idx))
            print(f"  ({courses[p_idx]:20}, {courses[c_idx]:20})")
            print(f"    Avantage de x : +{p_contrib} | Désavantage de x : {c_contrib} | Total : +{total}")
    
    # Vérification que tous les cons sont couverts
    covered_cons_indices = set()
    for p_idx, c_idx in explanation_pairs:
        covered_cons_indices.add(c_idx)
    
    print("\n" + "-" * 70)
    covered_courses = [courses[i] for i in sorted(covered_cons_indices)]
    print(f"Cours couverts (cons) : {', '.join(covered_courses)}")
    print(f"Tous les cons couverts : {'✓ OUI' if len(covered_cons_indices) == len(cons) else '✗ NON'}")


RÉSOLUTION DU PROGRAMME D'OPTIMISATION LINÉAIRE

Statut de résolution : OPTIMAL ✓

✓ UNE EXPLICATION DE TYPE (1-1) EXISTE
Longueur minimale de l'explication : 3 trade-offs

Explication détaillée:
----------------------------------------------------------------------
  (Anatomie            , Chirurgie           )
    Avantage de x : +32 | Désavantage de x : -28 | Total : +4
  (Diagnostic          , Forensic            )
    Avantage de x : +36 | Désavantage de x : -35 | Total : +1
  (Epidemiologie       , Genetique           )
    Avantage de x : +48 | Désavantage de x : -42 | Total : +6

----------------------------------------------------------------------
Cours couverts (cons) : Chirurgie, Forensic, Genetique
Tous les cons couverts : ✓ OUI


## Résumé de la Solution

### Pour l'exemple donné (Xavier vs Yvonne):

**pros(x,y)** = {Anatomie, Diagnostic, Epidemiologie}
**cons(x,y)** = {Chirurgie, Forensic, Genetique}
**neutral(x,y)** = {Biologie}

### Explication trouvée:
- **(Anatomie, Chirurgie)** : Xavier est meilleur de 4 points en Anatomie, plus mauvais de 4 points en Chirurgie → différence = 0 (mais les poids font que c'est +32-28=+4 ✓)
- **(Diagnostic, Forensic)** : Xavier est meilleur de 6 points en Diagnostic, plus mauvais de 7 points en Forensic → différence = -1 (mais les poids font +36-35=+1 ✓)
- **(Epidemiologie, Genetique)** : Xavier est meilleur de 8 points en Epidemiologie, plus mauvais de 7 points en Genetique → différence = +1 (et les poids font +48-42=+6 ✓)

### Interprétation:
Cette explication montre que chaque avantage de Xavier sur Yvonne pèse plus lourd que son désavantage correspondant, ce qui justifie pourquoi Xavier est mieux classé.

## Réponse Complète à la Question 1

### A) Calcul de pros(x,y) et cons(x,y)

En calculant les contributions pondérées pour chaque cours :
- **contribution(i) = w_i × (x_i - y_i)**

Résultats :
- **pros(x,y) = {Anatomie (+32), Diagnostic (+36), Epidemiologie (+48)}**
- **cons(x,y) = {Chirurgie (-28), Forensic (-35), Genetique (-42)}**
- **neutral(x,y) = {Biologie (0)}**

### B) Énumération des trade-offs (1-1)

Un trade-off (1, 1) valide (P, C) satisfait :
- P ∈ pros(x,y) et C ∈ cons(x,y)
- contribution(P) + contribution(C) > 0

Les 6 trade-offs valides sont :
1. (Anatomie, Chirurgie) : 32 + (-28) = 4 ✓
2. (Diagnostic, Chirurgie) : 36 + (-28) = 8 ✓
3. (Diagnostic, Forensic) : 36 + (-35) = 1 ✓
4. (Epidemiologie, Chirurgie) : 48 + (-28) = 20 ✓
5. (Epidemiologie, Forensic) : 48 + (-35) = 13 ✓
6. (Epidemiologie, Genetique) : 48 + (-42) = 6 ✓

### C) Solution du Programme Linéaire

**Explication (1-1) trouvée :** longueur 3

E = {(Anatomie, Chirurgie), (Diagnostic, Forensic), (Epidemiologie, Genetique)}

**Interprétation pour Yvonne:**
- Xavier est meilleur en **Anatomie** (+4 points pondérés) ce qui compense son retard en **Chirurgie**
- Xavier est meilleur en **Diagnostic** (+1 point pondéré) ce qui compense son retard en **Forensic**
- Xavier est meilleur en **Épidémiologie** (+6 points pondérés) ce qui compense son retard en **Génétique**

**Conclusion:** Xavier est mieux classé car chacun de ses avantages pèse plus lourd que son désavantage correspondant.

## Formulation Mathématique Complète du Programme Linéaire

### Définition du problème

Donnés :
- Deux candidats x et y avec notes sur m cours
- Poids w_i pour chaque cours i
- Contribution(i) = w_i × (x_i - y_i)
- Ensembles : pros(x,y), cons(x,y), neutral(x,y)
- Ensemble des trade-offs valides T = {(P,C) : P ∈ pros, C ∈ cons, contribution(P) + contribution(C) > 0}

### Programme Linéaire

**Variables de décision :**
$$x_{P,C} \in \{0, 1\} \quad \forall (P,C) \in T$$

**Contraintes :**

1. Couverture complète des cons :
$$\sum_{(P,C) \in T : C = c} x_{P,C} = 1 \quad \forall c \in \text{cons}(x,y)$$

2. Limitation des pros :
$$\sum_{(P,C) \in T : P = p} x_{P,C} \leq 1 \quad \forall p \in \text{pros}(x,y)$$

**Fonction objectif :**
$$\min \sum_{(P,C) \in T} x_{P,C}$$

### Interprétation

- **Contrainte 1** : Chaque reproche (cours dans cons) doit être justifié par exactement un avantage
- **Contrainte 2** : Chaque avantage ne peut servir qu'une seule fois pour justifier un reproche
- **Objectif** : Minimiser la longueur de l'explication (nombre de paires)

### Cas d'infaisabilité

Si le PL est **infaisable**, cela signifie qu'il n'existe pas d'explication (1-1), ce qui constitue un **certificat de non-existence**. Dans ce cas, on ne peut pas justifier x ≻ y par des arguments simples basés sur les contributions pondérées.

In [7]:
# Exemple bonus : Cas d'infaisabilité (certificat de non-existence)
print("\n" + "=" * 70)
print("EXEMPLE BONUS : TEST DE NON-EXISTENCE D'EXPLICATION")
print("=" * 70)

# Créons un cas où X et Y sont quasi-identiques mais X > Y globalement
# par une très petite marge, ce qui pourrait ne pas admettre d'explication (1-1)

courses2 = ['Cours1', 'Cours2', 'Cours3']
x_notes2 = np.array([80, 85, 75])
y_notes2 = np.array([80, 85, 76])  # Y meilleur en Cours3
weights2 = np.array([1, 1, 1])

contributions2 = weights2 * (x_notes2 - y_notes2)

print("\nNouveau cas test :")
print(f"x_notes  = {x_notes2}")
print(f"y_notes  = {y_notes2}")
print(f"weights  = {weights2}")
print(f"contributions = {contributions2}")

pros2 = [i for i in range(len(courses2)) if contributions2[i] > 0]
cons2 = [i for i in range(len(courses2)) if contributions2[i] < 0]

print(f"\npros = {[courses2[i] for i in pros2]}")
print(f"cons = {[courses2[i] for i in cons2]}")

# Trouver les trade-offs valides
valid_tradeoffs2 = []
for p_idx in pros2:
    for c_idx in cons2:
        sum_contrib = contributions2[p_idx] + contributions2[c_idx]
        if sum_contrib > 0:
            valid_tradeoffs2.append((p_idx, c_idx))
            
print(f"\nTrade-offs valides : {len(valid_tradeoffs2)}")
if valid_tradeoffs2:
    print("Explication possible ✓")
else:
    print("⚠️ CERTIFICAT DE NON-EXISTENCE : Aucun trade-off valide n'existe!")
    print("Il est impossible de justifier x ≻ y par des arguments (1-1) simples.")



EXEMPLE BONUS : TEST DE NON-EXISTENCE D'EXPLICATION

Nouveau cas test :
x_notes  = [80 85 75]
y_notes  = [80 85 76]
weights  = [1 1 1]
contributions = [ 0  0 -1]

pros = []
cons = ['Cours3']

Trade-offs valides : 0
⚠️ CERTIFICAT DE NON-EXISTENCE : Aucun trade-off valide n'existe!
Il est impossible de justifier x ≻ y par des arguments (1-1) simples.


## Résumé de la Solution

### ✓ Questions traitées dans ce notebook :

1. **Calcul de pros(x,y) et cons(x,y)** ✓
   - Identification de tous les cours favorisant x ou y
   - Résultat : pros = {Anatomie, Diagnostic, Epidemiologie}, cons = {Chirurgie, Forensic, Genetique}

2. **Énumération des trade-offs (1-1)** ✓
   - Identification de tous les trade-offs où contribution(P) + contribution(C) > 0
   - Résultat : 6 trade-offs valides trouvés

3. **Formulation du PL** ✓
   - Variables binaires : x[P,C] pour chaque trade-off
   - Contrainte de couverture : chaque cons doit être couvert
   - Contrainte de limitation : chaque pro peut être utilisé au maximum une fois
   - Objectif : minimiser le nombre de paires

4. **Implémentation avec Gurobi** ✓
   - Résolution du PL avec le solveur Gurobi
   - Explication optimale trouvée : E = {(Anatomie, Chirurgie), (Diagnostic, Forensic), (Epidemiologie, Genetique)}

5. **Certificat de non-existence** ✓
   - Démonstration avec un contre-exemple où aucune explication (1-1) n'existe

### Résultat Principal

**Une explication de type (1-1) de longueur minimale 3 existe pour justifier que Xavier ≻ Yvonne**, permettant à Yvonne de comprendre précisément pourquoi Xavier est mieux classé.